<a href="https://colab.research.google.com/github/saharaqc/ExtratEmailPython/blob/main/Copia_GITHUB_Extraer_email_de_pagina_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatización de búsqueda de sitios web y correos electrónicos con Python

## Instalar las librerías necesarias

In [ ]:
#!pip install pandas openpyxl googlesearch-python beautifulsoup4 requests

#!pip install googlesearch-python


In [ ]:
import pandas as pd
import requests
import re

# Función para buscar un sitio web usando la API de Google Custom Search
def search_website_api(query, api_key, cx):
    """
    Busca el sitio web de una empresa utilizando Google Custom Search API.
    Devuelve el enlace más relevante entre los primeros 10 resultados.
    """
    try:
        # Buscar con un máximo de 10 resultados
        url = f"https://www.googleapis.com/customsearch/v1?q={query}+&inurl:contact&key={api_key}&cx={cx}&num=10"
        response = requests.get(url)
        if response.status_code == 200:
            results = response.json()
            if 'items' in results:
                # Filtrar los resultados para encontrar el más relevante
                for item in results['items']:
                    link = item['link']
                    if any(keyword in link for keyword in ['contact', 'inicio', 'about']):
                        return link  # Devuelve un enlace relevante
                # Si no encuentra coincidencias, devuelve el primero
                return results['items'][0]['link']
        return None
    except Exception as e:
        print(f"Error en la búsqueda del sitio web: {e}")
        return None
# Función para extraer el email de las web encontradas
def extract_email_from_website(url):
    try:
        # Verificar si la URL contiene el esquema (http/https)
        if not url.startswith('http://') and not url.startswith('https://'):
            url = 'http://' + url

        urls_to_check = [url, url.rstrip('/') + '/contact']  # Añadir /contact como ruta adicional
        for current_url in urls_to_check:
            try:
                response = requests.get(current_url, timeout=10)
                if response.status_code == 200:
                    emails = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', response.text)
                    if emails:
                        return emails[0]  # Retorna el primer correo encontrado
            except Exception as e:
                print(f"Error al acceder a {current_url}: {e}")
        return None  # Si no se encuentra ningún correo, retorna None
    except Exception as e:
        print(f"Error al procesar la URL {url}: {e}")
        return None
# Función para guardar los resultados en un archivo Excel
def save_to_excel(data, filename):
    try:
        # Guardar los resultados en un archivo Excel
        output_df = pd.DataFrame(data, columns=['NOMBRE', 'TIPO', 'WEBSITE', 'EMAIL'])
        output_df.to_excel(output_filename, index=False)
        print(f"Resultados guardados en {output_filename}")
    except Exception as e:
        print(f"Error al guardar el archivo Excel: {e}")

# Variables de entrada
input_filename = "Tu fichero.xlsx"  # Archivo de entrada con una columna llamada 'NOMBRE' y otra llamada 'TIPO
output_filename = "Resultado_Emails.xlsx" # Archivo de Salida
api_key = ''  # API Key de Google Cloud Console
cx = ''  # ID del Custom Search Engine

data = []

# Ejecución del script
if __name__ == "__main__":
    try:
        # Leer el archivo de entrada
        nombres_df = pd.read_excel(input_filename)

        # Asegúrate de que las columnas 'NOMBRE' y 'TIPO' estén presentes
        if 'NOMBRE' not in nombres_df.columns or 'TIPO' not in nombres_df.columns:
            raise ValueError("El archivo de entrada debe contener las columnas 'NOMBRE' y 'TIPO'.")

        # Procesar cada fila del DataFrame
        for _, row in nombres_df.iterrows():
            name = row['NOMBRE']
            tipo = row['TIPO']

            # Concatenar TIPO al inicio del NOMBRE
            search_query = f"{tipo} {name}"
            print(f"Procesando búsqueda: {search_query}")

            # Buscar el sitio web y correo electrónico
            website = search_website_api(search_query, api_key, cx)
            email = extract_email_from_website(website) if website else None

            # Agregar la fila procesada a los datos
            data.append([name, tipo, website, email])
          # crear archivo de salida y guardar
        save_to_excel(data, output_filename)

    except Exception as e:
        print(f"Error en el proceso: {e}")

## Encontrar archivos y ficheros del libro:

In [ ]:
!ls


## Guardar y descargar el archivo en caso de perdida:

In [ ]:
#from google.colab import files
#files.download('Resultado_Emails.xlsx')
